In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/Users/macbookpro/Documents/predict_publications/publications_prediction'

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import joblib

In [4]:
test_data = pd.read_csv('/Users/macbookpro/Documents/predict_publications/publications_prediction/data/test_data.csv')
train_df = pd.read_csv('/Users/macbookpro/Documents/predict_publications/publications_prediction/data/train_data.csv')
validation_df = pd.read_csv('/Users/macbookpro/Documents/predict_publications/publications_prediction/data/validation_data.csv')

In [10]:
# Convert 'timestamp' to a datetime format
train_df['date'] = pd.to_datetime(train_df['timestamp'], unit='s')

# Extracting temporal features from the 'date' column
train_df['hour'] = train_df['date'].dt.hour
train_df['day'] = train_df['date'].dt.day
train_df['dayofweek'] = train_df['date'].dt.dayofweek
train_df['month'] = train_df['date'].dt.month

# Now, aggregate data based on 'timestamp', 'lat', and 'lon'
agg_columns = {
    'likescount': 'mean',
    'commentscount': 'mean',
    'symbols_cnt': 'mean',
    'words_cnt': 'mean',
    'hashtags_cnt': 'mean',
    'mentions_cnt': 'mean',
    'links_cnt': 'mean',
    'emoji_cnt': 'mean',
}

grouped_data = train_df.groupby(['timestamp', 'lon', 'lat', 'hour', 'day', 'dayofweek', 'month']).agg(agg_columns).reset_index()
grouped_data.head()

,timestamp,lon,lat,hour,day,dayofweek,month,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,1546300800,0.000000,0.000000,0,1,1,1,31.666667,1.666667,51.333333,2.000000,2.000000,0.0,0.0,0.000000
1,1546300800,30.136232,60.000054,0,1,1,1,52.000000,1.000000,28.000000,0.500000,2.000000,0.0,0.0,0.500000
2,1546300800,30.138478,59.835705,0,1,1,1,32.000000,0.333333,46.000000,2.333333,3.000000,0.0,0.0,1.333333
3,1546300800,30.142969,60.023627,0,1,1,1,77.666667,3.333333,34.666667,2.666667,0.666667,0.0,0.0,1.666667
4,1546300800,30.142969,60.030359,0,1,1,1,19.000000,3.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [12]:
grouped_data['publication_count'] = train_df.groupby(['timestamp', 'lon', 'lat']).size().values
grouped_data.head()

,timestamp,lon,lat,hour,day,dayofweek,month,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt,publication_count
0,1546300800,0.000000,0.000000,0,1,1,1,31.666667,1.666667,51.333333,2.000000,2.000000,0.0,0.0,0.000000,3
1,1546300800,30.136232,60.000054,0,1,1,1,52.000000,1.000000,28.000000,0.500000,2.000000,0.0,0.0,0.500000,2
2,1546300800,30.138478,59.835705,0,1,1,1,32.000000,0.333333,46.000000,2.333333,3.000000,0.0,0.0,1.333333,3
3,1546300800,30.142969,60.023627,0,1,1,1,77.666667,3.333333,34.666667,2.666667,0.666667,0.0,0.0,1.666667,3
4,1546300800,30.142969,60.030359,0,1,1,1,19.000000,3.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1


In [13]:
# Drop 'timestamp' as it's strongly correlated with other time features and may cause data leakage
X_train = grouped_data.drop(['publication_count', 'timestamp'], axis=1)
y_train = grouped_data['publication_count']

In [19]:
X_train.shape, y_train.shape

((3635541, 14), (3635541,))

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3635541 entries, 0 to 3635540
Data columns (total 14 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lon            float64
 1   lat            float64
 2   hour           int32  
 3   day            int32  
 4   dayofweek      int32  
 5   month          int32  
 6   likescount     float64
 7   commentscount  float64
 8   symbols_cnt    float64
 9   words_cnt      float64
 10  hashtags_cnt   float64
 11  mentions_cnt   float64
 12  links_cnt      float64
 13  emoji_cnt      float64
dtypes: float64(10), int32(4)
memory usage: 332.8 MB


In [14]:
# Convert the 'hour' column to a datetime format
test_data['date'] = pd.to_datetime(test_data['hour'], unit='s')

# Drop the original 'hour' column which contains the timestamp
test_data.drop(columns=['hour'], inplace=True)

# Extract the datetime features from the 'date' column
test_data['hour'] = test_data['date'].dt.hour
test_data['day'] = test_data['date'].dt.day
test_data['dayofweek'] = test_data['date'].dt.dayofweek
test_data['month'] = test_data['date'].dt.month

# Drop the 'date' column as it's not needed for prediction
test_data.drop(columns=['date'], inplace=True)

train_df_for_testing = pd.read_csv('/Users/macbookpro/Documents/predict_publications/publications_prediction/data/train_data.csv')

# Set 'point' as the index for both datasets
train_df_for_testing.set_index('point', inplace=True)
test_data.set_index('point', inplace=True)

# List of features to create in the test dataset
features_to_create = ['likescount', 'commentscount', 'symbols_cnt', 'words_cnt', 
                      'hashtags_cnt', 'mentions_cnt', 'links_cnt', 'emoji_cnt']

# Aggregate the training dataset based on 'point' and compute the median for each feature
aggregated_data = train_df_for_testing[features_to_create].groupby('point').median()

# Merge the test dataset with the aggregated training data on 'point'
test_data = test_data.join(aggregated_data, on='point', how='left')

# Reset index for both datasets after the operations
train_df_for_testing.reset_index(inplace=True)
test_data.reset_index(inplace=True)

X_test = test_data.drop(['sum', 'point', 'error'], axis=1)
y_test = test_data['sum']
X_test = X_test[X_train.columns]


In [16]:
X_test.shape, y_test.shape

((700, 14), (700,))

In [17]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lon            700 non-null    float64
 1   lat            700 non-null    float64
 2   hour           700 non-null    int32  
 3   day            700 non-null    int32  
 4   dayofweek      700 non-null    int32  
 5   month          700 non-null    int32  
 6   likescount     700 non-null    float64
 7   commentscount  700 non-null    float64
 8   symbols_cnt    700 non-null    float64
 9   words_cnt      700 non-null    float64
 10  hashtags_cnt   700 non-null    float64
 11  mentions_cnt   700 non-null    float64
 12  links_cnt      700 non-null    float64
 13  emoji_cnt      700 non-null    float64
dtypes: float64(10), int32(4)
memory usage: 65.8 KB


In [18]:
X_test.head()

,lon,lat,hour,day,dayofweek,month,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,30.331616,59.934863,10,26,2,2,44.0,1.0,73.0,4.0,0.0,0.0,0.0,1.0
1,30.329370,59.940488,11,17,0,2,41.0,1.0,47.0,2.0,0.0,0.0,0.0,0.0
2,30.297929,59.905597,16,12,2,2,28.0,0.0,42.0,2.0,0.0,0.0,0.0,0.0
3,30.356319,59.921358,13,12,2,2,48.0,1.0,110.0,6.0,0.0,0.0,0.0,1.0
4,30.315895,59.939363,13,15,5,2,47.0,1.0,49.0,2.0,0.0,0.0,0.0,0.0


# RandomForestRegressor

In [21]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=42)

In [22]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Predictions
rf_predictions = rf_model.predict(X_test)

# Evaluate the Random Forest model
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))

# Calculating the relative error for the Gradient Boosting model predictions
rf_relative_errors = np.abs(rf_predictions - y_test) / rf_predictions
rf_average_relative_error = rf_relative_errors.mean()


print(f"ARE: {rf_average_relative_error}")
print(f"RMSE: {rf_rmse}")

ARE: 7.1921547458456745
RMSE: 10.642302552301723


# GradientBoostingRegressor

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize and train the Gradient Boosting model
gb_model = GradientBoostingRegressor(n_estimators=100, max_depth=8, learning_rate=0.0895527640998049, subsample=0.5970487514167024, min_samples_split=20, min_samples_leaf=4, random_state=42)

gb_model.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.0895527640998049, max_depth=8,
                          min_samples_leaf=4, min_samples_split=20,
                          random_state=42, subsample=0.5970487514167024)

In [24]:
# Predict using the Gradient Boosting model
gb_predictions = gb_model.predict(X_test)  # gb_model is already loaded and trained

# Evaluate the Gradient Boosting model
gb_rmse = np.sqrt(mean_squared_error(y_test, gb_predictions))

# Calculating the relative error for the Gradient Boosting model predictions
gb_relative_errors = np.abs(gb_predictions - y_test) / gb_predictions
gb_average_relative_error = gb_relative_errors.mean()
gb_average_relative_error

6.441987344758175

# LSTM

In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam

# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshaping data for LSTM [samples, time steps, features]
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])


In [27]:
from keras.models import clone_model
from keras.callbacks import ModelCheckpoint

def create_lstm_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
    return model

best_model = None
lowest_val_loss = float('inf')

# Time Series Split
tscv = TimeSeriesSplit(n_splits=5)
for train_index, val_index in tscv.split(X_train_reshaped):
    X_train_fold, X_val_fold = X_train_reshaped[train_index], X_train_reshaped[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    model = create_lstm_model()
    
    # Use ModelCheckpoint to save the model with the lowest validation loss
    checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, validation_data=(X_val_fold, y_val_fold), callbacks=[checkpoint], verbose=2, shuffle=False)
    
    # Check if this fold's model is better than the previous best model
    val_loss = model.history.history['val_loss'][-1]
    if val_loss < lowest_val_loss:
        best_model = clone_model(model)
        best_model.load_weights('best_model.h5')
        best_model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
        lowest_val_loss = val_loss


Epoch 1/10
18936/18936 - 68s - loss: 6.0040 - val_loss: 15.3487 - 68s/epoch - 4ms/step
Epoch 2/10


/Users/macbookpro/miniconda3/envs/predict_publications_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18936/18936 - 37s - loss: 5.6181 - val_loss: 15.5636 - 37s/epoch - 2ms/step
Epoch 3/10
18936/18936 - 35s - loss: 5.5254 - val_loss: 15.3149 - 35s/epoch - 2ms/step
Epoch 4/10
18936/18936 - 32s - loss: 5.4839 - val_loss: 15.2680 - 32s/epoch - 2ms/step
Epoch 5/10
18936/18936 - 41s - loss: 5.4545 - val_loss: 15.3143 - 41s/epoch - 2ms/step
Epoch 6/10
18936/18936 - 33s - loss: 5.4114 - val_loss: 15.1145 - 33s/epoch - 2ms/step
Epoch 7/10
18936/18936 - 32s - loss: 5.1136 - val_loss: 13.8061 - 32s/epoch - 2ms/step
Epoch 8/10
18936/18936 - 31s - loss: 4.7541 - val_loss: 12.9232 - 31s/epoch - 2ms/step
Epoch 9/10
18936/18936 - 32s - loss: 4.5706 - val_loss: 12.1905 - 32s/epoch - 2ms/step
Epoch 10/10
18936/18936 - 35s - loss: 4.4449 - val_loss: 12.0020 - 35s/epoch - 2ms/step


Epoch 1/10
37871/37871 - 61s - loss: 9.9092 - val_loss: 19.7933 - 61s/epoch - 2ms/step
Epoch 2/10


/Users/macbookpro/miniconda3/envs/predict_publications_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

/Users/macbookpro/miniconda3/envs/predict_publications_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


75741/75741 - 110s - loss: 12.5507 - val_loss: 16.2462 - 110s/epoch - 1ms/step
Epoch 3/10
75741/75741 - 133s - loss: 10.6992 - val_loss: 14.9405 - 133s/epoch - 2ms/step
Epoch 4/10
75741/75741 - 100s - loss: 9.9339 - val_loss: 14.3591 - 100s/epoch - 1ms/step
Epoch 5/10
75741/75741 - 110s - loss: 9.5620 - val_loss: 13.8411 - 110s/epoch - 1ms/step
Epoch 6/10
75741/75741 - 111s - loss: 9.3858 - val_loss: 13.7843 - 111s/epoch - 1ms/step
Epoch 7/10
75741/75741 - 99s - loss: 9.2028 - val_loss: 13.7589 - 99s/epoch - 1ms/step
Epoch 8/10
75741/75741 - 102s - loss: 9.1306 - val_loss: 13.5464 - 102s/epoch - 1ms/step
Epoch 9/10
75741/75741 - 112s - loss: 9.0176 - val_loss: 13.4089 - 112s/epoch - 1ms/step
Epoch 10/10
75741/75741 - 102s - loss: 8.9312 - val_loss: 13.4283 - 102s/epoch - 1ms/step


Epoch 1/10


/Users/macbookpro/miniconda3/envs/predict_publications_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94676/94676 - 140s - loss: 13.8038 - val_loss: 24.9670 - 140s/epoch - 1ms/step
Epoch 2/10
94676/94676 - 123s - loss: 11.7782 - val_loss: 26.4387 - 123s/epoch - 1ms/step
Epoch 3/10
94676/94676 - 149s - loss: 10.7609 - val_loss: 43.0446 - 149s/epoch - 2ms/step
Epoch 4/10
94676/94676 - 127s - loss: 10.4327 - val_loss: 29.6376 - 127s/epoch - 1ms/step
Epoch 5/10
94676/94676 - 121s - loss: 10.2149 - val_loss: 20.0634 - 121s/epoch - 1ms/step
Epoch 6/10
94676/94676 - 127s - loss: 10.1091 - val_loss: 19.2332 - 127s/epoch - 1ms/step
Epoch 7/10
94676/94676 - 125s - loss: 9.9741 - val_loss: 18.0149 - 125s/epoch - 1ms/step
Epoch 8/10
94676/94676 - 145s - loss: 9.9051 - val_loss: 18.4289 - 145s/epoch - 2ms/step
Epoch 9/10
94676/94676 - 126s - loss: 9.8098 - val_loss: 18.2015 - 126s/epoch - 1ms/step
Epoch 10/10
94676/94676 - 152s - loss: 9.7854 - val_loss: 17.0211 - 152s/epoch - 2ms/step


In [33]:
# Reshape X_test for the LSTM model
X_test_reshaped = scaler.transform(X_test).reshape(X_test.shape[0], 1, X_test.shape[1])

# Predictions on the validation set
lstm_predictions = best_model.predict(X_test_reshaped).flatten()

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))
print(f"RMSE on the validation set: {rmse}")

# Calculate Average Relative Error using the provided formula
epsilon = 1e-10  # To ensure we're not dividing by zero
are = np.mean(np.abs(y_test - lstm_predictions) / (lstm_predictions + epsilon))
print(f"Average Relative Error on the validation set: {are}")

22/22 [==============================] - 0s 883us/step
RMSE on the validation set: 9.6844950802608
Average Relative Error on the validation set: 3.0718928774431853
